In [1]:
from pathlib import Path
import json

import pandas as pd
import tensorflow as tf
from keras.models import load_model

from data import constants
from data.eluc_data import ELUCData, ELUCEncoder
from prescriptors.esp.unileaf_prescriptor import UnileafPrescriptor, reco_to_context_actions
from predictors.neural_network.neural_net_predictor import NeuralNetPredictor

### Generate Seeds

In [2]:
def create_template_model():
    """
    Creates keras template prescriptor given architecture from paper:
        Input layer for each context variable
        Dense layer for each context variable hidden size 16
        Tanh activation
        Output as reco_land_use vector
    """
    inputs = [tf.keras.Input(shape=(1,), name=f"{col}_input") for col in constants.CAO_MAPPING["context"]]
    dense = [tf.keras.layers.Dense(16, name=constants.CAO_MAPPING["context"][i])(inputs[i]) for i in range(len(inputs))]
    add4 = tf.keras.layers.Add()(dense)
    activation = tf.keras.layers.Activation("tanh", name="first_hidden_activation")(add4)
    output = tf.keras.layers.Dense(len(constants.RECO_COLS), name="reco_land_use")(activation)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

def seed_no_change(data_df: pd.DataFrame, encoded_data_df: pd.DataFrame, seed_dir: Path):
    """
    Creates seed model that attempts to prescribe zero change.
    This is now feasible because we no longer softmax the output but instead linearly scale them.
    """

    no_change_preds = data_df[constants.RECO_COLS].copy()
    y_train = no_change_preds.to_numpy()
    X_train = [encoded_data_df[col].values for col in constants.CAO_MAPPING["context"]]

    no_change_model = create_template_model()
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
    no_change_model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mae'])
    no_change_model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=1)

    seed_dir.mkdir(parents=True, exist_ok=True)
    no_change_model.save(seed_dir / "1_1.h5")

def seed_max_change(data_df: pd.DataFrame, encoded_data_df: pd.DataFrame, seed_dir: Path, best_col="secdf"):
    """
    Creates seed model that attempts to prescribe maximum change.
    Moves all possible land use to best_col which is secdf by default.
    """
    # Move all the land use to secdf
    land_use = data_df[constants.RECO_COLS].sum(axis=1)
    max_change_preds = data_df[constants.RECO_COLS].copy()
    max_change_preds[constants.RECO_COLS] = 0
    max_change_preds[best_col] = land_use

    y_train = max_change_preds.to_numpy()
    X_train = [encoded_data_df[col].values for col in constants.CAO_MAPPING["context"]]
    
    max_change_model = create_template_model()
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
    max_change_model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mae'])
    max_change_model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=1)

    seed_dir.mkdir(parents=True, exist_ok=True)
    max_change_model.save(seed_dir / "1_2.h5")

In [3]:
dataset = ELUCData()

In [4]:
# Take small subset for training, we really don't need more and just need the model to converge
seed_sample = dataset.train_df.sample(1000, random_state=42)
encoded_seed_sample = dataset.encoder.encode_as_df(seed_sample)
seed_dir = Path("prescriptors/esp/seeds/test")

In [5]:
seed_no_change(seed_sample, encoded_seed_sample, seed_dir)
seed_max_change(seed_sample, encoded_seed_sample, seed_dir)

Epoch 1/300
8/8 [==============================] - 1s 28ms/step - loss: 0.3318 - mae: 0.3318
Epoch 2/300
8/8 [==============================] - 0s 13ms/step - loss: 0.2603 - mae: 0.2603
Epoch 3/300
8/8 [==============================] - 0s 12ms/step - loss: 0.2447 - mae: 0.2447
Epoch 4/300
8/8 [==============================] - 0s 12ms/step - loss: 0.2291 - mae: 0.2291
Epoch 5/300
8/8 [==============================] - 0s 12ms/step - loss: 0.2131 - mae: 0.2131
Epoch 6/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1996 - mae: 0.1996
Epoch 7/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1863 - mae: 0.1863
Epoch 8/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1743 - mae: 0.1743
Epoch 9/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1632 - mae: 0.1632
Epoch 10/300
8/8 [==============================] - 0s 15ms/step - loss: 0.1523 - mae: 0.1523
Epoch 11/300
8/8 [==============================] - 0s 12ms/step - lo

/Users/964840/anaconda3/envs/leaf/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 25ms/step - loss: 0.4574 - mae: 0.4574
Epoch 2/300
8/8 [==============================] - 0s 12ms/step - loss: 0.3186 - mae: 0.3186
Epoch 3/300
8/8 [==============================] - 0s 12ms/step - loss: 0.2623 - mae: 0.2623
Epoch 4/300
8/8 [==============================] - 0s 15ms/step - loss: 0.2409 - mae: 0.2409
Epoch 5/300
8/8 [==============================] - 0s 12ms/step - loss: 0.2269 - mae: 0.2269
Epoch 6/300
8/8 [==============================] - 0s 13ms/step - loss: 0.2119 - mae: 0.2119
Epoch 7/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1970 - mae: 0.1970
Epoch 8/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1844 - mae: 0.1844
Epoch 9/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1720 - mae: 0.1720
Epoch 10/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1602 - mae: 0.1602
Epoch 11/300
8/8 [==============================] - 0s 12ms/step - loss: 0.1491 -

/Users/964840/anaconda3/envs/leaf/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Evaluate Seeds

In [6]:
def trained_prescribe_and_predict(encoded_context: pd.DataFrame, 
                                  prescriptor_path: Path, 
                                  encoder: ELUCEncoder, 
                                  dummy_prescriptor: UnileafPrescriptor) -> pd.DataFrame:
    """
    Loads a prescriptor model, prescribes land use for context, predicts ELUC and calculates change.
    """
    prescriptor = load_model(prescriptor_path, compile=False)
    reco_land_use = dummy_prescriptor.prescribe(prescriptor, encoded_context)
    reco_df = pd.DataFrame(reco_land_use["reco_land_use"].tolist(), columns=constants.RECO_COLS)
    context_actions_df = reco_to_context_actions(reco_df, encoded_context, encoder)
    prescribed_actions_df = context_actions_df[constants.DIFF_LAND_USE_COLS]
    eluc = dummy_prescriptor.predict_eluc(context_actions_df)
    change = (prescribed_actions_df[prescribed_actions_df > 0].sum(axis=1) / context_actions_df[constants.LAND_USE_COLS].sum(axis=1))

    context_actions_df["ELUC"] = eluc.values
    context_actions_df["change"] = change.values

    return context_actions_df

In [7]:
nnp = NeuralNetPredictor()
nnp.load("predictors/neural_network/trained_models/no_overlap_nn")

In [8]:
presc_config = None
with open("prescriptors/esp/unileaf_configs/config-loctime-crop-nosoft.json", "r") as f:
    presc_config = json.load(f)
dummy_prescriptor = UnileafPrescriptor(presc_config,
                                   dataset.train_df.iloc[:1],
                                   dataset.encoder,
                                   [nnp])

In [9]:
test_df = dataset.test_df.sample(frac=0.01, random_state=100)
encoded_test_df = dataset.encoder.encode_as_df(test_df)
context_df = test_df[constants.CAO_MAPPING["context"]]
encoded_context_df = encoded_test_df[constants.CAO_MAPPING["context"]]

In [10]:
result_df = trained_prescribe_and_predict(encoded_context_df, seed_dir / "1_1.h5", dataset.encoder, dummy_prescriptor)
print(f"Average ELUC of no change prescriptor on test set: {result_df['ELUC'].mean()}")
print(f"Average change of no change prescriptor on test set: {result_df['change'].mean()}")

832/832 [==============================] - 3s 3ms/step
Average ELUC of no change prescriptor on test set: 0.061359677324486044
Average change of no change prescriptor on test set: 0.011037839944058702


In [11]:
result_df = trained_prescribe_and_predict(encoded_context_df, seed_dir / "1_2.h5", dataset.encoder, dummy_prescriptor)
print(f"Average ELUC of max change prescriptor on test set: {result_df['ELUC'].mean()}")
print(f"Average change of max change prescriptor on test set: {result_df['change'].mean()}")

832/832 [==============================] - 3s 3ms/step
Average ELUC of max change prescriptor on test set: -24.228178468619262
Average change of max change prescriptor on test set: 0.4239889862987677
